https://www.kaggle.com/chocozzz/xgboost-tutorial 참조함. 

In [6]:
import numpy as np
import pandas as pd
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import re

In [7]:
%matplotlib inline

In [9]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

all_data = pd.concat([train, test], sort=False)
all_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [10]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
all_data.Embarked = le.fit_transform(all_data.Embarked.fillna('0'))
all_data.Sex = le.fit_transform(all_data.Sex.fillna('3'))
all_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,3
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,1
2,3,1.0,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,3
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,3
4,5,0.0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,3


In [11]:
# split the data back into train and test

df_train = all_data.loc[all_data.Survived.isin([np.nan]) == False]
df_test = all_data.loc[all_data.Survived.isin([np.nan]) == True]

In [12]:
print(df_train.shape)
print(df_test.shape)
df_train.head()

(891, 12)
(418, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,3
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,1
2,3,1.0,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,3
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,3
4,5,0.0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,3


In [13]:
df_test.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,NaN,3,"Kelly, Mr. James",1,34.5,0,0,330911,7.8292,NaN,2
1,893,NaN,3,"Wilkes, Mrs. James (Ellen Needs)",0,47.0,1,0,363272,7.0000,NaN,3
2,894,NaN,2,"Myles, Mr. Thomas Francis",1,62.0,0,0,240276,9.6875,NaN,2
3,895,NaN,3,"Wirz, Mr. Albert",1,27.0,0,0,315154,8.6625,NaN,3
4,896,NaN,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,22.0,1,1,3101298,12.2875,NaN,3


In [14]:
'''
# XGBoost model + parameter tuning with GridSearchCV
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, train_test_split

feature_names = ['Sex','Embarked','Pclass','Survived','Age','SibSp','Parch','Fare']

xgb = XGBRegressor()
params={
    'max_depth': [2,3,4,5], 
    'subsample': [0.4,0.5,0.6,0.7,0.8,0.9,1.0],
    'colsample_bytree': [0.5,0.6,0.7,0.8],
    'n_estimators': [1000,2000,3000],
    'reg_alpha': [0.01, 0.02, 0.03, 0.04]
}

grs = GridSearchCV(xgb, param_grid=params, cv = 10, n_jobs=4, verbose=2)
grs.fit(np.array(df_train[feature_names]), np.array(df_train['Survived']))

print("Best parameters " + str(grs.best_params_))
gpd = pd.DataFrame(grs.cv_results_)
print("Estimated accuracy of this model for unseen data: {0:1.4f}".format(gpd['mean_test_score'][grs.best_index_]))
# TODO: why is this so bad?
'''

'\n# XGBoost model + parameter tuning with GridSearchCV\nfrom xgboost import XGBRegressor\nfrom sklearn.model_selection import GridSearchCV, train_test_split\n\nfeature_names = [\'Sex\',\'Embarked\',\'Pclass\',\'Survived\',\'Age\',\'SibSp\',\'Parch\',\'Fare\']\n\nxgb = XGBRegressor()\nparams={\n    \'max_depth\': [2,3,4,5], \n    \'subsample\': [0.4,0.5,0.6,0.7,0.8,0.9,1.0],\n    \'colsample_bytree\': [0.5,0.6,0.7,0.8],\n    \'n_estimators\': [1000,2000,3000],\n    \'reg_alpha\': [0.01, 0.02, 0.03, 0.04]\n}\n\ngrs = GridSearchCV(xgb, param_grid=params, cv = 10, n_jobs=4, verbose=2)\ngrs.fit(np.array(df_train[feature_names]), np.array(df_train[\'Survived\']))\n\nprint("Best parameters " + str(grs.best_params_))\ngpd = pd.DataFrame(grs.cv_results_)\nprint("Estimated accuracy of this model for unseen data: {0:1.4f}".format(gpd[\'mean_test_score\'][grs.best_index_]))\n# TODO: why is this so bad?\n'

위에는 그리드서치라는 기법을 이용해서 예측한건데 여기서 선택지가 3개로 갈림.

그리드 서치 2. 랜덤 서치 3. 베이지안 최적화
그리드 < 랜덤 < 베이지안 으로 괜찮은 방법입니다.

Best parameters {'colsample_bytree': 0.8, 'max_depth': 2, 'n_estimators': 3000, 'reg_alpha': 0.01, 'subsample': 0.6}

In [15]:
train_y = df_train.Survived
train_x = df_train.drop('Survived', axis=1)

excluded_feats = ['PassengerId', "Ticket", 'Cabin', 'Name']
features = [f_ for f_ in train_x.columns if f_ not in excluded_feats]
features

['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

In [16]:
from sklearn.model_selection import KFold

folds = KFold(n_splits=4, shuffle=True, random_state=546789)

In [17]:
oof_preds = np.zeros(train_y.shape[0])
sub_preds = np.zeros(df_test.shape[0])

아래는 XGBOOST 모델 만드는 부분인데 정보손실 최소화 하기 위해서 kfold방식 이용했습니다.

순서는

X변수와 Y변수를 분리해준다.

모델의 하이퍼파라미터를 조정해준다. 여기에서 위에서 말한 그리드서치, 랜덤서치, 베이지안 최적화가 쓰입니다.
베이지안 최적화 in XGBOOST : http://www.kwangsiklee.com/2018/06/bayesianoptimization%EC%9D%84-%EC%9D%B4%EC%9A%A9%ED%95%98%EC%97%AC-xgboost-%ED%95%98%EC%9D%B4%ED%8D%BC-%ED%8C%8C%EB%9D%BC%EB%AF%B8%ED%84%B0%EB%A5%BC-%EC%B5%9C%EC%A0%81%ED%99%94/

In [19]:
import gc
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix


for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train_x)):

    trn_x, trn_y = train_x[features].iloc[trn_idx], train_y.iloc[trn_idx]

    val_x, val_y = train_x[features].iloc[val_idx], train_y.iloc[val_idx]

    

    clf = XGBClassifier(

        objective = 'binary:logistic', #문제에 맞게. 

        booster = "gbtree",

        eval_metric = 'auc', 

        nthread = 4,

        eta = 0.05,

        gamma = 0,

        max_depth = 2, 

        subsample = 0.6, 

        colsample_bytree = 0.8, 

        colsample_bylevel = 0.675,

        min_child_weight = 22,

        alpha = 0,

        random_state = 42, 

        nrounds = 2000,
        
        n_estimators=3000

    )



    clf.fit(trn_x, trn_y, eval_set= [(trn_x, trn_y), (val_x, val_y)], verbose=10, early_stopping_rounds=100)

    

    oof_preds[val_idx] = clf.predict_proba(val_x)[:, 1]

    sub_preds += clf.predict_proba(df_test[features])[:, 1] / folds.n_splits

    

    print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(val_y, oof_preds[val_idx])))

    del clf, trn_x, trn_y, val_x, val_y

    gc.collect()

    

print('Full AUC score %.6f' % roc_auc_score(train_y, oof_preds))   



test['Survived'] = sub_preds



test[['PassengerId', 'Survived']].to_csv('xgb_submission_esi.csv', index=False, float_format='%.8f')

[0]	validation_0-auc:0.789759	validation_1-auc:0.812331
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 100 rounds.
[10]	validation_0-auc:0.839629	validation_1-auc:0.856491
[20]	validation_0-auc:0.83914	validation_1-auc:0.859576
[30]	validation_0-auc:0.842411	validation_1-auc:0.860167
[40]	validation_0-auc:0.84215	validation_1-auc:0.860167
[50]	validation_0-auc:0.840801	validation_1-auc:0.860167
[60]	validation_0-auc:0.844728	validation_1-auc:0.860717
[70]	validation_0-auc:0.847272	validation_1-auc:0.861984
[80]	validation_0-auc:0.849167	validation_1-auc:0.8649
[90]	validation_0-auc:0.84817	validation_1-auc:0.865196
[100]	validation_0-auc:0.849271	validation_1-auc:0.866802
[110]	validation_0-auc:0.849281	validation_1-auc:0.867224
[120]	validation_0-auc:0.848692	validation_1-auc:0.866886
[130]	validation_0-auc:0.846755	validation_1-auc:0.86714
[140]	validation_0-auc:0.848939	validation_1-au